In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

In [ ]:
# Base path
base_path = "/kaggle/input/formula-1-world-championship-1950-2020/"

# List of CSV filenames
files = [
    "circuits.csv",
    "constructor_results.csv",
    "constructor_standings.csv",
    "constructors.csv",
    "driver_standings.csv",
    "drivers.csv",
    "lap_times.csv",
    "pit_stops.csv",
    "qualifying.csv",
    "races.csv",
    "results.csv",
    "seasons.csv",
    "sprint_results.csv",
    "status.csv"
]


In [ ]:
# Load each CSV file into a variable
circuits = pd.read_csv(base_path + "circuits.csv")
constructor_results = pd.read_csv(base_path + "constructor_results.csv")
constructor_standings = pd.read_csv(base_path + "constructor_standings.csv")
constructors = pd.read_csv(base_path + "constructors.csv")
driver_standings = pd.read_csv(base_path + "driver_standings.csv")
drivers = pd.read_csv(base_path + "drivers.csv")
lap_times = pd.read_csv(base_path + "lap_times.csv")
pit_stops = pd.read_csv(base_path + "pit_stops.csv")
qualifying = pd.read_csv(base_path + "qualifying.csv")
races = pd.read_csv(base_path + "races.csv")
results = pd.read_csv(base_path + "results.csv")
seasons = pd.read_csv(base_path + "seasons.csv")
sprint_results = pd.read_csv(base_path + "sprint_results.csv")
status = pd.read_csv(base_path + "status.csv")

In [ ]:
# Print columns and row count for each dataframe
print("=== CIRCUITS ===")
print(f"Rows: {len(circuits)}")
print(circuits.columns.tolist())

print("\n=== CONSTRUCTOR_RESULTS ===")
print(f"Rows: {len(constructor_results)}")
print(constructor_results.columns.tolist())

print("\n=== CONSTRUCTOR_STANDINGS ===")
print(f"Rows: {len(constructor_standings)}")
print(constructor_standings.columns.tolist())

print("\n=== CONSTRUCTORS ===")
print(f"Rows: {len(constructors)}")
print(constructors.columns.tolist())

print("\n=== DRIVER_STANDINGS ===")
print(f"Rows: {len(driver_standings)}")
print(driver_standings.columns.tolist())

print("\n=== DRIVERS ===")
print(f"Rows: {len(drivers)}")
print(drivers.columns.tolist())

print("\n=== LAP_TIMES ===")
print(f"Rows: {len(lap_times)}")
print(lap_times.columns.tolist())

print("\n=== PIT_STOPS ===")
print(f"Rows: {len(pit_stops)}")
print(pit_stops.columns.tolist())

print("\n=== QUALIFYING ===")
print(f"Rows: {len(qualifying)}")
print(qualifying.columns.tolist())

print("\n=== RACES ===")
print(f"Rows: {len(races)}")
print(races.columns.tolist())

print("\n=== RESULTS ===")
print(f"Rows: {len(results)}")
print(results.columns.tolist())

print("\n=== SEASONS ===")
print(f"Rows: {len(seasons)}")
print(seasons.columns.tolist())

print("\n=== SPRINT_RESULTS ===")
print(f"Rows: {len(sprint_results)}")
print(sprint_results.columns.tolist())

print("\n=== STATUS ===")
print(f"Rows: {len(status)}")
print(status.columns.tolist())

In [ ]:
print("=== CIRCUITS ===")
print(circuits.iloc[0])

print("\n=== CONSTRUCTOR_RESULTS ===")
print(constructor_results.iloc[0])

print("\n=== CONSTRUCTOR_STANDINGS ===")
print(constructor_standings.iloc[0])

print("\n=== CONSTRUCTORS ===")
print(constructors.iloc[0])

print("\n=== DRIVER_STANDINGS ===")
print(driver_standings.iloc[0])

print("\n=== DRIVERS ===")
print(drivers.iloc[0])

print("\n=== LAP_TIMES ===")
print(lap_times.iloc[0])

print("\n=== PIT_STOPS ===")
print(pit_stops.iloc[0])

print("\n=== QUALIFYING ===")
print(qualifying.iloc[0])

print("\n=== RACES ===")
print(races.iloc[0])

print("\n=== RESULTS ===")
print(results.iloc[0])

print("\n=== SEASONS ===")
print(seasons.iloc[0])

print("\n=== SPRINT_RESULTS ===")
print(sprint_results.iloc[0])

print("\n=== STATUS ===")
print(status.iloc[0])


In [ ]:
recent_races = races[(races['year'] >= 2018) & (races['year'] <= 2024)]
recent_race_ids = recent_races['raceId'].unique()


In [ ]:
# Apply filter to relevant datasets
driver_standings_recent = driver_standings[driver_standings['raceId'].isin(recent_race_ids)]
constructor_standings_recent = constructor_standings[constructor_standings['raceId'].isin(recent_race_ids)]
results_recent = results[results['raceId'].isin(recent_race_ids)]
lap_times_recent = lap_times[lap_times['raceId'].isin(recent_race_ids)]
pit_stops_recent = pit_stops[pit_stops['raceId'].isin(recent_race_ids)]
qualifying_recent = qualifying[qualifying['raceId'].isin(recent_race_ids)]


In [ ]:
# Step 1: Basic (raceId, driverId) pair from results — this will be our base
base_df = results_recent[['raceId', 'driverId', 'constructorId', 'grid', 'positionOrder', 'points', 'statusId']].copy()
base_df.rename(columns={'positionOrder': 'final_position'}, inplace=True)

# Step 2: Merge with driver standings
base_df = base_df.merge(driver_standings_recent[['raceId', 'driverId', 'points', 'wins']],
                        on=['raceId', 'driverId'],
                        how='left',
                        suffixes=('', '_driver_standing'))

base_df.rename(columns={
    'points': 'points_driver_standing',
    'wins': 'wins_driver_standing'
}, inplace=True)

# Step 3: Merge with constructor standings
base_df = base_df.merge(constructor_standings_recent[['raceId', 'constructorId', 'points', 'wins']],
                        on=['raceId', 'constructorId'],
                        how='left',
                        suffixes=('', '_constructor_standing'))

base_df.rename(columns={
    'points': 'points_constructor_standing',
    'wins': 'wins_constructor_standing'
}, inplace=True)

# Step 4: Aggregate lap times per driver per race
lap_agg = lap_times_recent.groupby(['raceId', 'driverId']).agg({
    'milliseconds': ['mean', 'min', 'max', 'std'],
    'lap': 'count'
}).reset_index()
lap_agg.columns = ['raceId', 'driverId', 'lap_ms_mean', 'lap_ms_min', 'lap_ms_max', 'lap_ms_std', 'total_laps']

base_df = base_df.merge(lap_agg, on=['raceId', 'driverId'], how='left')

# Step 5: Aggregate pit stop data
pit_agg = pit_stops_recent.groupby(['raceId', 'driverId']).agg({
    'milliseconds': ['count', 'mean', 'sum']
}).reset_index()
pit_agg.columns = ['raceId', 'driverId', 'num_pitstops', 'avg_pit_duration', 'total_pit_duration']

base_df = base_df.merge(pit_agg, on=['raceId', 'driverId'], how='left')

# Step 6: Qualifying performance (best time in any session as proxy)
def get_best_qual_time(row):
    for q in ['q1', 'q2', 'q3']:
        if pd.notnull(row[q]):
            return row[q]
    return None

qualifying_recent['best_qual_time'] = qualifying_recent.apply(get_best_qual_time, axis=1).copy()

# Convert best_qual_time to seconds
def time_to_seconds(t):
    try:
        m, s = t.split(':')
        return float(m) * 60 + float(s)
    except:
        return None

qualifying_recent['best_qual_seconds'] = qualifying_recent['best_qual_time'].apply(time_to_seconds)

qual_data = qualifying_recent[['raceId', 'driverId', 'position', 'best_qual_seconds']]
base_df = base_df.merge(qual_data, on=['raceId', 'driverId'], how='left')

# Final preprocessing: fill missing values
base_df.fillna(-1, inplace=True)

# Check sample
print(base_df.head())
print("Total rows in training dataset:", len(base_df))


In [ ]:
# Assuming 'races' dataframe is loaded already
race_years = races[['raceId', 'year']]
base_df = base_df.merge(race_years, on='raceId', how='left')

In [ ]:
train_df = base_df[(base_df['year'] >= 2018) & (base_df['year'] <= 2023)]
test_df = base_df[base_df['year'] == 2024]

print(f"Training data rows: {train_df.shape[0]}")
print(f"Test data rows: {test_df.shape[0]}")


In [ ]:
features = [
    'grid', 'statusId',
    'points_driver_standing', 'wins_driver_standing',
    'points_constructor_standing', 'wins_constructor_standing',
    'lap_ms_mean', 'lap_ms_min', 'lap_ms_max', 'lap_ms_std', 'total_laps',
    'num_pitstops', 'avg_pit_duration', 'total_pit_duration',
    'position',  # qualifying position
    'best_qual_seconds'
]


X_train = train_df[features]
y_train = train_df['final_position']

X_test = test_df[features]


In [ ]:
train_df = train_df.loc[:, ~train_df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Define and train the model
model = XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    random_state=42
)
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)


In [ ]:
test_df['predicted_position'] = y_pred
test_df['predicted_position'] = y_pred.astype(int)


In [ ]:
test_df_sorted = test_df.sort_values(['raceId', 'predicted_position'])


In [ ]:
for race_id, group in test_df_sorted.groupby('raceId'):
    print(f"\nRace {race_id} Predicted Results:")
    print(group[['driverId', 'constructorId', 'grid', 'predicted_position']].head(10))


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_df['final_position'], test_df['predicted_position'])
print("MAE on 2024 season:", mae)


In [ ]:
print(test_df.columns)
print(drivers.columns)

In [ ]:
# Make sure 'code' column doesn't already exist to avoid conflict
if 'code' in test_df.columns:
    test_df = test_df.drop(columns=['code'])

# Merge driver code
test_df = test_df.merge(drivers[['driverId', 'code']], on='driverId', how='left')

# Merge races with circuits
races = races.merge(circuits[['circuitId', 'name']], on='circuitId', how='left')
races.rename(columns={'name': 'circuit_name'}, inplace=True)

# Merge circuit name into test_df using raceId
test_df = test_df.merge(races[['raceId', 'circuit_name']], on='raceId', how='left')

# Convert predicted_position to int
test_df['predicted_position'] = test_df['predicted_position'].astype(int)

# Display results for 2024
test_df_sorted = test_df.sort_values(['raceId', 'predicted_position'])
for race_id, group in test_df_sorted[test_df_sorted['year'] == 2024].groupby('raceId'):
    circuit = group['circuit_name'].iloc[0]
    print(f"\n=== 🏁 RACE ID: {race_id} | CIRCUIT: {circuit} ===")
    top10 = group.sort_values('predicted_position').head(10)[['code', 'grid', 'predicted_position', 'final_position']]
    print(top10.rename(columns={'code': 'Driver', 'grid': 'Grid', 'predicted_position': 'Predicted', 'final_position': 'Actual'}).to_string(index=False))
